# 4일차 4교시 조인

### 목차
* 1. 조인 유형
* 2. Inner Join
* 3. Outer Join
* 4. 조인 유의사항

### 참고 사이트
* [PySpark Search](https://spark.apache.org/docs/latest/api/python/search.html)
* [Pyspark Functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?#module-pyspark.sql.functions)

### 1. 조인 유형

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession \
    .builder \
    .config("spark.sql.session.timeZone", "Asia/Seoul") \
    .getOrCreate()

### JOIN 학습을 위해 상품은 단 하나만 구매할 수 있다고 가정하여 아래와 같은 테이블이 존재합니다
#### 정보 1. 고객은 4명이지만, 1명은 탈퇴하여 존재하지 않습니다
| 고객 아이디 (u_id) | 고객 이름 (u_name) | 고객 성별 (u_gender) |
| - | - | - |
| 1 | 정휘센 | 남 |
| 2 | 김싸이언 | 남 |
| 3 | 박트롬 | 여 |

#### 정보 2. 구매 상품은 3개이며, 탈퇴한 고객의 상품정보가 남아있습니다
| 구매 고객 아이디 (u_id) | 구매 상품 이름 (p_name) | 구매 상품 가격 (p_amount) |
| - | - | - |
| 2 | LG DIOS | 2,000,000 |
| 3 | LG Cyon | 1,800,000 |
| 4 | LG Computer | 4,500,000 |


In [3]:
user = spark.createDataFrame([
    (1, "정휘센", "남"),
    (2, "김싸이언", "남"),
    (3, "박트롬", "여")
]).toDF("u_id", "u_name", "u_gender")
user.printSchema()
user.show()
    
purchase = spark.createDataFrame([
    (2, "LG DIOS", 2000000),
    (3, "LG Cyon", 1800000),
    (4, "LG Computer", 4500000)
]).toDF("p_uid", "p_name", "p_amont")
purchase.printSchema()
purchase.show()

root
 |-- u_id: long (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)

+----+--------+--------+
|u_id|  u_name|u_gender|
+----+--------+--------+
|   1|  정휘센|      남|
|   2|김싸이언|      남|
|   3|  박트롬|      여|
+----+--------+--------+

root
 |-- p_uid: long (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amont: long (nullable = true)

+-----+-----------+-------+
|p_uid|     p_name|p_amont|
+-----+-----------+-------+
|    2|    LG DIOS|2000000|
|    3|    LG Cyon|1800000|
|    4|LG Computer|4500000|
+-----+-----------+-------+



## 2. Inner Join
### 2.1 구매 정보와 일치하는 고객 정보를 조인 (inner)

In [4]:
user.join(purchase, user.u_id == purchase.p_uid).show()
user.join(purchase, user.u_id == purchase.p_uid, "inner").count()

+----+--------+--------+-----+-------+-------+
|u_id|  u_name|u_gender|p_uid| p_name|p_amont|
+----+--------+--------+-----+-------+-------+
|   3|  박트롬|      여|    3|LG Cyon|1800000|
|   2|김싸이언|      남|    2|LG DIOS|2000000|
+----+--------+--------+-----+-------+-------+



2

## 3. Outer Join
### 3.1 모든 고객의 정보 구매 정보를 조인 (left_outer)

In [5]:
user.join(purchase, user.u_id == purchase.p_uid, "left_outer").orderBy(purchase.p_uid.asc()).show()

+----+--------+--------+-----+-------+-------+
|u_id|  u_name|u_gender|p_uid| p_name|p_amont|
+----+--------+--------+-----+-------+-------+
|   1|  정휘센|      남| null|   null|   null|
|   2|김싸이언|      남|    2|LG DIOS|2000000|
|   3|  박트롬|      여|    3|LG Cyon|1800000|
+----+--------+--------+-----+-------+-------+



### 3-2. 모든 상품에 대한 고객 정보를 조인 (right_outer)

In [6]:
user.join(purchase, user.u_id == purchase.p_uid, "right_outer").orderBy(purchase.p_uid.asc()).show()

+----+--------+--------+-----+-----------+-------+
|u_id|  u_name|u_gender|p_uid|     p_name|p_amont|
+----+--------+--------+-----+-----------+-------+
|   2|김싸이언|      남|    2|    LG DIOS|2000000|
|   3|  박트롬|      여|    3|    LG Cyon|1800000|
|null|    null|    null|    4|LG Computer|4500000|
+----+--------+--------+-----+-----------+-------+



### 3-3. 모든 고객과 상품에 대한 정보를 조인 (full_outer)

In [7]:
user.join(purchase, user.u_id == purchase.p_uid, "full_outer").orderBy(purchase.p_uid.asc()).show()

+----+--------+--------+-----+-----------+-------+
|u_id|  u_name|u_gender|p_uid|     p_name|p_amont|
+----+--------+--------+-----+-----------+-------+
|   1|  정휘센|      남| null|       null|   null|
|   2|김싸이언|      남|    2|    LG DIOS|2000000|
|   3|  박트롬|      여|    3|    LG Cyon|1800000|
|null|    null|    null|    4|LG Computer|4500000|
+----+--------+--------+-----+-----------+-------+



## 4. 조인시 유의할 점


In [8]:
u = spark.createDataFrame([
    (1, "정휘센", "남"),
    (2, "김싸이언", "남"),
    (3, "박트롬", "여")
]).toDF("id", "name", "gender")
u.printSchema()
u.show()
    
p = spark.createDataFrame([
    (2, "LG DIOS", 2000000),
    (3, "LG Cyon", 1800000),
    (4, "LG Computer", 4500000)
]).toDF("id", "name", "amount")
p.printSchema()
p.show()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)

+---+--------+------+
| id|    name|gender|
+---+--------+------+
|  1|  정휘센|    남|
|  2|김싸이언|    남|
|  3|  박트롬|    여|
+---+--------+------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- amount: long (nullable = true)

+---+-----------+-------+
| id|       name| amount|
+---+-----------+-------+
|  2|    LG DIOS|2000000|
|  3|    LG Cyon|1800000|
|  4|LG Computer|4500000|
+---+-----------+-------+



### 3.1 중복 컬럼명 처리가 되지 않은 경우
> #### AnalysisException: "Reference 'id' is ambiguous, could be: id, id.;"

In [9]:
up = u.join(p, u.id == p.id)
up.show()
# up.select("id")

+---+--------+------+---+-------+-------+
| id|    name|gender| id|   name| amount|
+---+--------+------+---+-------+-------+
|  3|  박트롬|    여|  3|LG Cyon|1800000|
|  2|김싸이언|    남|  2|LG DIOS|2000000|
+---+--------+------+---+-------+-------+



### 3.2 중복 컬럼명 해결방안 - 데이터 프레임의 컬럼 명을 다르게 만든다

In [10]:
u1 = u.withColumnRenamed("id", "u_uid")
p1 = p.withColumnRenamed("id", "p_uid")
u1.printSchema()
p1.printSchema()

up = u1.join(p1, u1.u_uid == p1.p_uid)
up.show()
up.select("u_uid")

root
 |-- u_uid: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)

root
 |-- p_uid: long (nullable = true)
 |-- name: string (nullable = true)
 |-- amount: long (nullable = true)

+-----+--------+------+-----+-------+-------+
|u_uid|    name|gender|p_uid|   name| amount|
+-----+--------+------+-----+-------+-------+
|    3|  박트롬|    여|    3|LG Cyon|1800000|
|    2|김싸이언|    남|    2|LG DIOS|2000000|
+-----+--------+------+-----+-------+-------+



DataFrame[u_uid: bigint]

### 3.3 중복 컬럼명 해결방안 - 조인 직후 중복 컬럼을 제거합니다

In [11]:
up = u.join(p, u.id == p.id).drop(p.id)
up.show()
up.select("id")

+---+--------+------+-------+-------+
| id|    name|gender|   name| amount|
+---+--------+------+-------+-------+
|  3|  박트롬|    여|LG Cyon|1800000|
|  2|김싸이언|    남|LG DIOS|2000000|
+---+--------+------+-------+-------+



DataFrame[id: bigint]

### 실습#7 data/tbl_purchase.csv 과 data/tbl_user.csv 파일을 읽고, 상품을 구매한 고객에 대해서만 이름과 상품을 같이 출력하세요 (inner join을 사용하세요)
> 참고: user.join(purchase, condition, "inner")

In [12]:
purchase = spark.read.option("header", "true").option("inferSchema", "true").csv("data/tbl_purchase.csv")
user = spark.read.option("header", "true").option("inferSchema", "true").csv("data/tbl_user.csv")

user.join(purchase, user.u_id == purchase.p_uid, "inner").show()

purchase.printSchema()
user.printSchema()

+----+----------+--------+--------+----------+-----+----+-----------+--------+
|u_id|    u_name|u_gender|u_signup|    p_time|p_uid|p_id|     p_name|p_amount|
+----+----------+--------+--------+----------+-----+----+-----------+--------+
|   1|    정휘센|      남|19580808|1603694755|    1|2000|    LG Gram| 1800000|
|   1|    정휘센|      남|19580808|1603651550|    1|2000|    LG DIOS| 2000000|
|   2|  김싸이언|      남|19590201|1603673500|    2|2001|    LG Cyon| 1400000|
|   3|    박트롬|      여|19951030|1603652155|    3|2002|      LG TV| 1000000|
|   4|    청소기|      남|19770329|1603674500|    4|2003|LG Computer| 4500000|
|   5|유코드제로|      여|20021029|1603666155|    5|2004|      LG TV| 2500000|
|   5|유코드제로|      여|20021029|1603665955|    5|2004|    LG Gram| 3500000|
+----+----------+--------+--------+----------+-----+----+-----------+--------+

root
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p

### 실습#8 data/tbl_purchase.csv 과 data/tbl_user.csv 파일을 읽고, 모든 고객의 이름과 상품을 같이 출력하세요 (left outer join을 사용하세요)
> 참고: user.join(purchase, condition, "left")

In [13]:
purchase = spark.read.option("header", "true").option("inferSchema", "true").csv("data/tbl_purchase.csv")
user = spark.read.option("header", "true").option("inferSchema", "true").csv("data/tbl_user.csv")

user.join(purchase, user.u_id == purchase.p_uid, "left_outer").show()

purchase.printSchema()
user.printSchema()

+----+----------+--------+--------+----------+-----+----+-----------+--------+
|u_id|    u_name|u_gender|u_signup|    p_time|p_uid|p_id|     p_name|p_amount|
+----+----------+--------+--------+----------+-----+----+-----------+--------+
|   1|    정휘센|      남|19580808|1603694755|    1|2000|    LG Gram| 1800000|
|   1|    정휘센|      남|19580808|1603651550|    1|2000|    LG DIOS| 2000000|
|   2|  김싸이언|      남|19590201|1603673500|    2|2001|    LG Cyon| 1400000|
|   3|    박트롬|      여|19951030|1603652155|    3|2002|      LG TV| 1000000|
|   4|    청소기|      남|19770329|1603674500|    4|2003|LG Computer| 4500000|
|   5|유코드제로|      여|20021029|1603666155|    5|2004|      LG TV| 2500000|
|   5|유코드제로|      여|20021029|1603665955|    5|2004|    LG Gram| 3500000|
|   6|  윤디오스|      남|20040101|      null| null|null|       null|    null|
|   7|  임모바일|      남|20040807|      null| null|null|       null|    null|
|   8|  조노트북|      여|20161201|      null| null|null|       null|    null|
|   9|  최컴퓨터|      남|